# 深度学习第一课：一元线性回归（手写反向传播）

## 为什么从这里开始？

一元线性回归是所有深度学习模型的"祖宗"。虽然简单，但它包含了深度学习的**全部核心要素**：

| 核心概念 | 在线性回归中的体现 |
|---------|------------------|
| 参数 | 权重 $w$ 和偏置 $b$ |
| 损失函数 | 均方误差 MSE |
| 梯度 | $\frac{\partial L}{\partial w}$ 和 $\frac{\partial L}{\partial b}$ |
| 优化 | 梯度下降 |

最重要的是：**每一步都能手算、手推、手写**。如果这一步没吃透，后面所有"深度"都是空的。

---
## 1. 环境准备

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
np.random.seed(42)

---
## 2. 生成训练数据

我们人工构造一组数据，其真实规律为：

$$y = 2x + 3 + \epsilon, \quad \epsilon \sim N(0, \sigma^2)$$

其中：
- **真实权重** $w_{true} = 2$
- **真实偏置** $b_{true} = 3$  
- $\epsilon$ 是高斯噪声，模拟真实世界中的观测误差

### 为什么用人工数据？

因为**真值已知**，我们可以：
1. 验证学习算法是否正确（最终参数应接近真值）
2. 观察 Loss 是否真的在下降
3. 通过调整噪声大小，理解"不可约误差"

In [ ]:
# 真实参数
w_true = 2.0
b_true = 3.0

# 生成数据
x = np.linspace(-10, 10, 100)
noise_std = 1.0
y = w_true * x + b_true + np.random.randn(100) * noise_std

print(f"数据点数量: {len(x)}")
print(f"真实参数: w = {w_true}, b = {b_true}")

### 数据可视化

下图展示了生成的数据点（蓝色散点）和真实的线性函数（红色虚线）。

我们的目标是：**仅从蓝色散点，推断出红色虚线的参数**。

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(x, y, alpha=0.6, label='观测数据', color='dodgerblue', s=30)
plt.plot(x, w_true * x + b_true, 'r--', linewidth=2, label=f'真实函数 y={w_true}x+{b_true}')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

---
## 3. 核心公式推导

### 3.1 前向传播 (Forward Pass)

给定参数 $w$ 和 $b$，模型的预测值为：

$$\hat{y} = w \cdot x + b$$

### 3.2 损失函数 (Loss Function)

我们使用**均方误差 (MSE)** 来衡量预测值与真实值的差距：

$$L = \frac{1}{N} \sum_{i=1}^{N} (y_i - \hat{y}_i)^2$$

> **核心洞察**：Loss 就是"误差能量"。它越小，说明我们的预测越准确。

### 3.3 反向传播 (Backward Pass) — 梯度计算

为了最小化 Loss，我们需要知道：**参数往哪个方向调整，Loss 会下降？**

答案是**梯度的反方向**。

令 $e_i = y_i - \hat{y}_i$ (误差)，通过链式法则推导：

$$\frac{\partial L}{\partial w} = \frac{\partial}{\partial w} \left[ \frac{1}{N} \sum e_i^2 \right] = \frac{2}{N} \sum e_i \cdot \frac{\partial e_i}{\partial w} = -\frac{2}{N} \sum x_i \cdot e_i$$

$$\frac{\partial L}{\partial b} = -\frac{2}{N} \sum e_i$$

> **核心洞察**：梯度是 Loss 增加最快的方向。我们朝梯度的**反方向**走，Loss 就会下降。

### 3.4 梯度下降 (Gradient Descent)

有了梯度，通过以下规则迭代更新参数：

$$w \leftarrow w - \eta \cdot \frac{\partial L}{\partial w}$$
$$b \leftarrow b - \eta \cdot \frac{\partial L}{\partial b}$$

其中 $\eta$ 是**学习率 (learning rate)**，控制每次更新的步长大小。

---
## 4. 代码实现

下面我们将上述公式翻译成代码，完整实现一个梯度下降的线性回归。

In [ ]:
# 超参数
learning_rate = 0.01
num_epochs = 100

# 随机初始化参数
w = np.random.randn()
b = np.random.randn()

# 记录训练历史
history = {'loss': [], 'w': [], 'b': []}
N = len(x)

print(f"初始参数: w = {w:.4f}, b = {b:.4f}")
print(f"目标参数: w = {w_true}, b = {b_true}")
print("-" * 50)

In [ ]:
for epoch in range(num_epochs):
    # 前向传播
    y_pred = w * x + b
    
    # 计算损失
    loss = ((y - y_pred) ** 2).mean()
    
    # 反向传播：计算梯度
    dw = (-2 / N) * np.sum(x * (y - y_pred))
    db = (-2 / N) * np.sum(y - y_pred)
    
    # 梯度下降：更新参数
    w = w - learning_rate * dw
    b = b - learning_rate * db
    
    # 记录历史
    history['loss'].append(loss)
    history['w'].append(w)
    history['b'].append(b)
    
    if epoch % 20 == 0 or epoch == num_epochs - 1:
        print(f"Epoch {epoch:3d} | Loss: {loss:.4f} | w: {w:.4f} | b: {b:.4f}")

In [ ]:
print("-" * 50)
print(f"最终学习到的参数: w = {w:.4f}, b = {b:.4f}")
print(f"真实参数:         w = {w_true}, b = {b_true}")
print(f"参数误差:         Δw = {abs(w - w_true):.4f}, Δb = {abs(b - b_true):.4f}")

---
## 5. 训练过程可视化

### 5.1 Loss 下降曲线

下图展示了 Loss 随训练轮次的变化。可以观察到 Loss 逐渐下降并趋于稳定，这说明模型正在学习。

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Loss 曲线
axes[0].plot(history['loss'], color='crimson', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss (MSE)')
axes[0].set_title('Loss 下降曲线')
axes[0].set_yscale('log')
axes[0].grid(True, alpha=0.3)

# w 的变化
axes[1].plot(history['w'], color='forestgreen', linewidth=2, label='学习到的 w')
axes[1].axhline(y=w_true, color='red', linestyle='--', linewidth=2, label=f'真实值 w={w_true}')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('w')
axes[1].set_title('权重 w 变化')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# b 的变化  
axes[2].plot(history['b'], color='purple', linewidth=2, label='学习到的 b')
axes[2].axhline(y=b_true, color='red', linestyle='--', linewidth=2, label=f'真实值 b={b_true}')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('b')
axes[2].set_title('偏置 b 变化')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 5.2 拟合效果对比

下图对比了真实函数（红色虚线）和学习到的函数（绿色实线）。可以看到两条线几乎完全重合。

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(x, y, alpha=0.6, label='训练数据', color='dodgerblue', s=30)
plt.plot(x, w_true * x + b_true, 'r--', linewidth=2.5, label=f'真实函数 y={w_true}x+{b_true}')
plt.plot(x, w * x + b, 'g-', linewidth=2.5, label=f'学习结果 y={w:.2f}x+{b:.2f}')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### 5.3 参数空间中的优化轨迹

下图展示了在参数空间 $(w, b)$ 中的优化轨迹：
- **等高线**：表示不同参数组合对应的 Loss 值（越深色 Loss 越小）
- **红色轨迹**：参数从初始点到最终点的移动路径
- **黄色星**：起点
- **绿色星**：终点
- **红色 X**：真实参数位置

这张图直观展示了梯度下降的本质：沿着 Loss 下降最快的方向，逐步逼近最优解。

In [ ]:
# 创建参数网格
w_range = np.linspace(-1, 4, 100)
b_range = np.linspace(-1, 6, 100)
W, B = np.meshgrid(w_range, b_range)

# 计算 Loss 曲面
Loss_surface = np.zeros_like(W)
for i in range(W.shape[0]):
    for j in range(W.shape[1]):
        y_pred_grid = W[i, j] * x + B[i, j]
        Loss_surface[i, j] = ((y - y_pred_grid) ** 2).mean()

# 绘图
plt.figure(figsize=(10, 8))
contour = plt.contour(W, B, Loss_surface, levels=30, cmap='viridis', alpha=0.8)
plt.colorbar(contour, label='Loss (MSE)')

plt.plot(history['w'], history['b'], 'ro-', markersize=3, linewidth=1.5, label='优化轨迹', alpha=0.8)
plt.scatter(history['w'][0], history['b'][0], s=200, c='yellow', edgecolors='black', marker='*', zorder=5, label='起点')
plt.scatter(history['w'][-1], history['b'][-1], s=200, c='lime', edgecolors='black', marker='*', zorder=5, label='终点')
plt.scatter(w_true, b_true, s=200, c='red', edgecolors='white', marker='X', zorder=5, label='真实值')

plt.xlabel('w')
plt.ylabel('b')
plt.title('参数空间中的优化轨迹')
plt.legend(loc='upper right')
plt.show()

---
## 6. 实验：动手探索

以下实验帮助你更深入理解各个概念。**强烈建议亲自修改参数并运行**。

### 实验 1：学习率的影响

学习率 $\eta$ 控制每次参数更新的步长：
- **太小**：收敛很慢，需要很多轮训练
- **适中**：收敛快且稳定
- **太大**：可能震荡甚至发散（Loss 不降反升）

下面对比三种学习率的效果：

In [ ]:
def train_with_lr(lr, num_epochs=50):
    """使用指定学习率训练，返回 Loss 历史"""
    w_exp, b_exp = 0.5, 0.5  # 固定起点
    losses = []
    
    for _ in range(num_epochs):
        y_pred = w_exp * x + b_exp
        loss = ((y - y_pred) ** 2).mean()
        losses.append(loss)
        
        dw = (-2 / N) * np.sum(x * (y - y_pred))
        db = (-2 / N) * np.sum(y - y_pred)
        
        w_exp = w_exp - lr * dw
        b_exp = b_exp - lr * db
        
        if loss > 1e10:
            break
    
    return losses

learning_rates = [0.001, 0.01, 0.05]
colors = ['blue', 'green', 'red']

plt.figure(figsize=(10, 5))
for lr, color in zip(learning_rates, colors):
    losses = train_with_lr(lr)
    plt.plot(losses, label=f'lr = {lr}', color=color, linewidth=2)

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('不同学习率的 Loss 曲线对比')
plt.legend()
plt.yscale('log')
plt.grid(True, alpha=0.3)
plt.show()

### 实验 2：噪声的影响

噪声 $\sigma$ 控制数据的随机性：
- **$\sigma = 0$**：无噪声，学习到的参数应该完全等于真值
- **$\sigma$ 增大**：噪声增大，学习到的参数会偏离真值

**思考问题**：噪声变大后，最优解还应该等于 $(w=2, b=3)$ 吗？

In [ ]:
noise_levels = [0.0, 1.0, 3.0, 5.0]
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for idx, noise in enumerate(noise_levels):
    np.random.seed(42)
    y_noisy = w_true * x + b_true + np.random.randn(100) * noise
    
    # 训练
    w_n, b_n = 0.0, 0.0
    for _ in range(200):
        y_pred = w_n * x + b_n
        dw = (-2 / N) * np.sum(x * (y_noisy - y_pred))
        db = (-2 / N) * np.sum(y_noisy - y_pred)
        w_n = w_n - 0.01 * dw
        b_n = b_n - 0.01 * db
    
    # 绘图
    axes[idx].scatter(x, y_noisy, alpha=0.5, s=20)
    axes[idx].plot(x, w_true * x + b_true, 'r--', linewidth=2, label='真实函数')
    axes[idx].plot(x, w_n * x + b_n, 'g-', linewidth=2, label='学习结果')
    axes[idx].set_title(f'噪声 σ={noise}, 学习到 w={w_n:.2f}, b={b_n:.2f}')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

**问题的答案**：

理论上，当数据量足够大时，最优解的期望值仍然等于真值 $(2, 3)$，因为噪声是零均值的。

但在实际中（有限样本），噪声会导致学习到的参数产生偏差。这就是所谓的**方差 (variance)** ——噪声越大，估计的不确定性越大。

### 实验 3：为什么 Loss 降了但 w 没有完全等于 2？

这是一个关键问题。让我们来分析：

In [ ]:
def compute_loss(w_val, b_val):
    y_pred = w_val * x + b_val
    return ((y - y_pred) ** 2).mean()

# 真实参数的 Loss
loss_true = compute_loss(w_true, b_true)

# 学习到的参数的 Loss
loss_learned = compute_loss(w, b)

# 解析解（最小二乘法的闭式解）
w_analytical = np.sum((x - x.mean()) * (y - y.mean())) / np.sum((x - x.mean())**2)
b_analytical = y.mean() - w_analytical * x.mean()
loss_analytical = compute_loss(w_analytical, b_analytical)

print("三种参数对应的 Loss 对比：")
print("-" * 50)
print(f"真实参数   (w={w_true:.4f}, b={b_true:.4f}) 的 Loss: {loss_true:.6f}")
print(f"学习参数   (w={w:.4f}, b={b:.4f}) 的 Loss: {loss_learned:.6f}")
print(f"解析解     (w={w_analytical:.4f}, b={b_analytical:.4f}) 的 Loss: {loss_analytical:.6f}")

**关键洞察**：

1. **真实参数不一定是 Loss 最小的点**。因为数据有噪声，最优拟合线会略微偏向噪声。

2. **解析解（最小二乘）才是 Loss 真正最小的点**。梯度下降会逼近解析解，而不是真实参数。

3. **只有当噪声为 0 时，最优解才完全等于真实参数**。

---
## 7. 动画演示

下面的动画展示了训练过程中拟合线的实时变化和 Loss 的下降过程。

In [ ]:
from IPython.display import HTML
from matplotlib.animation import FuncAnimation

# 重新训练，保存中间状态
np.random.seed(123)
w_anim, b_anim = 0.0, 0.0
lr_anim = 0.02
frames_w, frames_b, frames_loss = [w_anim], [b_anim], []

for _ in range(60):
    y_pred = w_anim * x + b_anim
    loss = ((y - y_pred) ** 2).mean()
    frames_loss.append(loss)
    
    dw = (-2 / N) * np.sum(x * (y - y_pred))
    db = (-2 / N) * np.sum(y - y_pred)
    
    w_anim = w_anim - lr_anim * dw
    b_anim = b_anim - lr_anim * db
    
    frames_w.append(w_anim)
    frames_b.append(b_anim)

# 创建动画
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.scatter(x, y, alpha=0.5, s=20)
ax1.plot(x, w_true * x + b_true, 'r--', linewidth=2, label='真实函数')
learned_line, = ax1.plot([], [], 'g-', linewidth=2.5, label='当前拟合')
ax1.set_xlim(-12, 12)
ax1.set_ylim(-25, 30)
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.legend()
ax1.grid(True, alpha=0.3)
title1 = ax1.set_title('')

loss_line, = ax2.plot([], [], 'crimson', linewidth=2)
ax2.set_xlim(0, 60)
ax2.set_ylim(0, max(frames_loss) * 1.1)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.set_title('Loss 下降曲线')
ax2.grid(True, alpha=0.3)

def animate(i):
    y_fit = frames_w[i] * x + frames_b[i]
    learned_line.set_data(x, y_fit)
    title1.set_text(f'Epoch {i}: w={frames_w[i]:.3f}, b={frames_b[i]:.3f}')
    loss_line.set_data(range(i), frames_loss[:i])
    return learned_line, title1, loss_line

anim = FuncAnimation(fig, animate, frames=len(frames_w)-1, interval=100, blit=False)
plt.close()
HTML(anim.to_jshtml())

---
## 8. 总结

### 核心概念检查清单

完成这个练习后，请确保你能回答以下问题：

| 问题 | 答案要点 |
|------|----------|
| Loss 是什么？ | "误差能量"，衡量预测与真实的差距 |
| 梯度是什么？ | Loss 相对于参数的变化率，指向 Loss 增加最快的方向 |
| 为什么用负梯度？ | 要让 Loss 下降，必须朝梯度反方向走 |
| 学习率的作用？ | 控制更新步长；太大震荡，太小收敛慢 |
| 训练的本质？ | 迭代调整参数，不断减小 Loss |

### 深度学习的核心循环

无论模型多复杂（CNN、RNN、Transformer），核心流程永远是：

```
前向传播 → 计算损失 → 反向求梯度 → 更新参数
```

你现在已经从最简单的线性回归中，完整理解了这个过程。接下来可以挑战：

1. **多元线性回归**：输入从 1 维变成多维
2. **逻辑回归**：用于分类问题
3. **多层感知机**：添加隐藏层和非线性激活函数